In [1]:
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
model = "gpt-4o-mini"

In [2]:
from openai import OpenAI

client = OpenAI()

In [3]:
messages = [
    {"role": "system", "content": (
        "You are a fact checker. Verify the validity of the sentence provided by the user, given a reference. You must return a response in JSON format:"
        "{'isFactTrue': <true or false>, 'explanation': <explanation> ")
    }
]

In [4]:
%pip install -q requests bs4
import requests
from bs4 import BeautifulSoup

def get_text_from_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.get_text()
    else:
        return "Error: Unable to retrieve content"


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.3/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_text_from_url",
            "description": "Get the text contents of a url",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "The url",
                    },
                },
                "required": ["url"],
            },
        },
    }
]

In [6]:
# 1
import json

# 2
def process_response_with_tools(response):
  # 3
  response_message = response.choices[0].message
  tool_calls = response_message.tool_calls

  # 4
  if tool_calls:
    # 5
    messages.append(response_message) 
    
    # 6
    available_functions = {
        "get_text_from_url": get_text_from_url,
    } 
    # 7
    for tool_call in tool_calls:
        # 8
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            url=function_args.get("url"),
        )
        # 9
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
        # 10
        tool_response = client.chat.completions.create(
            model=model,
            messages=messages,
            response_format={"type": "json_object"},
        )
        messages.append({"role": "assistant", "content": tool_response.choices[0].message.content})

        return tool_response
  # 11
  else:
    messages.append({"role": "assistant", "content": response.choices[0].message.content})
    return None


In [7]:
while True:
    user_input = input("Please enter your input: ")
 
    if user_input.lower() == 'exit':
        break

    messages.append({"role": "user", "content": user_input})

    try:
        response = client.chat.completions.create(
            model=model,
            tools=tools,
            messages=messages,
            response_format={"type": "json_object"},
        ) 

        tool_response = process_response_with_tools(response)
        if tool_response:
            print(tool_response.choices[0].message.content)
        else:
            print(response.choices[0].message.content)
    except openai.RateLimitError as e:
        print(f"Rate limit exceeded: {e}")

Please enter your input:  Flutter is the most popular framework https://www.statista.com/statistics/869224/worldwide-software-developer-working-hours/#:~:text=Flutter%20is%20the%20most%20popular,of%20software%20developers%20used%20Flutter.


{
  "isFactTrue": true,
  "explanation": "According to a 2023 survey, Flutter is reported as the most popular cross-platform mobile framework used by global developers, with 46 percent of software developers indicating they utilize Flutter."
}


Please enter your input:  Compare it to React Native and others


{
  "isFactTrue": true,
  "explanation": {
    "Flutter": {
      "usagePercentage": "46%",
      "trends": [
        {"year": 2019, "percentage": "30%"},
        {"year": 2020, "percentage": "39%"},
        {"year": 2021, "percentage": "42%"},
        {"year": 2022, "percentage": "46%"},
        {"year": 2023, "percentage": "46%"}
      ]
    },
    "React_Native": {
      "usagePercentage": "35%",
      "trends": [
        {"year": 2019, "percentage": "42%"},
        {"year": 2020, "percentage": "42%"},
        {"year": 2021, "percentage": "38%"},
        {"year": 2022, "percentage": "32%"},
        {"year": 2023, "percentage": "35%"}
      ]
    },
    "Other_Frameworks": {
      "Cordova": {
        "usagePercentage": "10%",
        "trends": [
          {"year": 2019, "percentage": "29%"},
          {"year": 2020, "percentage": "18%"},
          {"year": 2021, "percentage": "16%"},
          {"year": 2022, "percentage": "10%"},
          {"year": 2023, "percentage": "10%"}
       

Please enter your input:  exit


In [8]:
print(messages)

[{'role': 'system', 'content': "You are a fact checker. Verify the validity of the sentence provided by the user, given a reference. You must return a response in JSON format:{'isFactTrue': <true or false>, 'explanation': <explanation> "}, {'role': 'user', 'content': 'Flutter is the most popular framework https://www.statista.com/statistics/869224/worldwide-software-developer-working-hours/#:~:text=Flutter%20is%20the%20most%20popular,of%20software%20developers%20used%20Flutter.'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_t1J62PpKqFwx8boLQIL0DoEe', function=Function(arguments='{"url":"https://www.statista.com/statistics/869224/worldwide-software-developer-working-hours/#:~:text=Flutter%20is%20the%20most%20popular,of%20software%20developers%20used%20Flutter."}', name='get_text_from_url'), type='function')], refusal=None), {'tool_call_id': 'call_t1J62PpKqFwx8boLQIL0DoEe', 'role': 'tool', 'name': 'get_text_